In [1]:
import os
import sys
import networkx as nx
import pickle
from collections import defaultdict
#import matplotlib.pyplot as plt
import random
import community

In [2]:
class Status :
    node2com = {}
    total_weight = 0
    internals = {}
    degrees = {}
    gdegrees = {}

    layer={}
    node_l={}
    node_c={}       
    top={}
    bot={}
    edge_l={}
    edge_c={}
    couple={}
    mu = 0

    in_layer_in_comm ={}
    in_layer_out_comm ={}
    out_layer_in_comm ={}
    out_layer_out_comm ={}

    def __init__(self) :
        self.node2com = dict([])
        self.total_weight = 0
        self.degrees = dict([])
        self.gdegrees = dict([])
        self.internals = dict([])
        self.loops = dict([])

        self.layer=dict([])
        self.node_l=dict([])
        self.node_c=dict([])
        self.top=dict([])
        self.bot=dict([])
        self.edge_l=dict([])
        self.edge_c=dict([])
        self.couple=dict([])
        self.mu = 0

        self.in_layer_in_comm   = dict()
        self.in_layer_out_comm  = dict()
        self.out_layer_in_comm  = dict()
        self.out_layer_out_comm = dict()

    def __str__(self) :
        return ("node2com : " + str(self.node2com) + " degrees : "
            + str(self.degrees) + " internals : " + str(self.internals)
            + " total_weight : " + str(self.total_weight)) 

    def copy(self) :
        """Perform a deep copy of status"""
        new_status = Status()
        new_status.node2com = self.node2com.copy()
        new_status.internals = self.internals.copy()
        new_status.degrees = self.degrees.copy()
        new_status.gdegrees = self.gdegrees.copy()
        new_status.total_weight = self.total_weight
        new_status.layer=self.layer.copy()
        new_status.node_l=self.node_l.copy()
        new_status.node_c=self.node_c.copy()
        new_status.top=self.top.copy()
        new_status.bot=self.bot.copy()
        new_status.edge_l=self.edge_l.copy()
        new_status.edge_c=self.edge_c.copy()
        new_status.couple=self.couple.copy()
        new_status.mu = self.mu
        new_status.in_layer_in_comm = self.in_layer_in_comm.copy()
        new_status.in_layer_out_comm = self.in_layer_out_comm.copy()
        new_status.out_layer_in_comm = self.out_layer_in_comm.copy()
        new_status.out_layer_out_comm = self.out_layer_out_comm.copy()
        return new_status

    def updatelists(self, graph):
        self.in_layer_in_comm   = dict()
        self.in_layer_out_comm  = dict()
        self.out_layer_in_comm  = dict()
        self.out_layer_out_comm = dict()

        node2layer = dict()
        for l in self.layer:     #nodeset is the set of nodes in each layer
            nodeset = self.layer[l]
            for node in nodeset:
                node2layer[node] = l
        #print("node2layer: ",node2layer)
        #update in_layer_in_comm
        datakey = 'weight'                  #this is the key used to get edge weight from weightdict

        for node in graph.nodes():
            node_neighbours = graph[node]  #node_neighbours will be a dict {2: {'weight':3}, 3: {'weight':4}}
            self.in_layer_in_comm[node] = 0
            self.in_layer_out_comm[node] = 0
            self.out_layer_in_comm[node] = 0
            self.out_layer_out_comm[node] = 0
            
            for dest,edge_data in node_neighbours.items():
                edge_weight = edge_data.get("weight", 1)
                if(self.node2com[dest] == self.node2com[node]): #both nodes are in the same community
                    if(node2layer[node] == node2layer[dest]): #both nodes in same layer
                        self.in_layer_in_comm[node] += edge_weight
                        if(node == dest): self.in_layer_in_comm[node] += edge_weight     #Add self loop twice
                    else:
                        self.out_layer_in_comm[node] += edge_weight

                else:                                 #both nodes are in different community
                    if(node2layer[node] == node2layer[dest]): #both nodes in same layer
                        self.in_layer_out_comm[node] += edge_weight
                    else:
                        self.out_layer_out_comm[node] += edge_weight
        #print("In layer in comm: ",self.in_layer_in_comm)


    def init(self, graph, part = None) :

        """Initialize the status of a graph with every node in one community"""
        count = 1
        #count = 0
        self.node2com = dict([])
        self.total_weight = 0
        self.degrees = dict([])
        self.gdegrees = dict([])
        self.internals = dict([])
        self.total_weight = graph.size(weight = 'weight')

        if part == None :
            for node in graph.nodes() :
                self.node2com[node] = count
                deg = float(graph.degree(node, weight = 'weight'))
                if deg < 0 :
                    raise ValueError("Bad graph type, use positive weights")
                self.degrees[count] = deg
                self.gdegrees[node] = deg
                self.loops[node] = float(graph.get_edge_data(node, node,
                                                 {"weight":0}).get("weight", 1))
                self.internals[count] = self.loops[node]
                count = count + 1
        else :
            for node in graph.nodes() :
                com = part[node]
                self.node2com[node] = com
                deg = float(graph.degree(node, weight = 'weight'))
                self.degrees[com] = self.degrees.get(com, 0) + deg
                self.gdegrees[node] = deg
                inc = 0.
                for neighbor, datas in graph[node].items() :
                    weight = datas.get("weight", 1)
                    if weight <= 0 :
                        raise ValueError("Bad graph type, use positive weights")
                    if part[neighbor] == com :
                        if neighbor == node :
                            inc += float(weight)
                        else :
                            inc += float(weight) / 2.
                self.internals[com] = self.internals.get(com, 0) + inc

        self.updatelists(graph)


In [3]:
from collections import defaultdict

def _get_com_wise_nodes(dictionary):
    #m = max(dictionary.values())
    louvain_p = defaultdict(set)
    for l in dictionary.keys():
        louvain_p[dictionary[l]].add(l)
    return louvain_p

def printsomeinformation(node, com_node, best_node, incr,node_l,node_c):
    pass; return;
    print("Printing Information: {0} {1} {2} {3} {4} {5}".format(node, com_node, best_node, incr,node_l,node_c))



In [4]:
modctr = 0

def __modularityprint(commu, status, graph):
    global modctr
    modctr += 1
    #print("modularity called", modctr, "edgewt: ", [graph[1][nbr].get('weight',1) for nbr in graph[1]])
    #print("From modularity, node_c: ", status.node_c)
    #print("From modularity, node_l: ", status.node_l)

    layer=status.layer
    node_l=status.node_l
    node_c=status.node_c       
    top=status.top
    bot=status.bot
    edge_l=status.edge_l
    edge_c=status.edge_c
    couple=status.couple
    mu = status.mu
    
    #Construct a dict = {node1: layer, node2: layer......}
    nodelayer = {}
    for l in layer:
        for node in layer[l]:
            nodelayer[node] = l

    #calculate Intra_inter ----------------------------------------------------------
    f=0    
    intra_inter={}
    for c in commu:
        intra_inter[c]=set()
        
        for n in commu[c]:
            for l in layer:
                if n in layer[l]:
                    intra_inter[c].add(l)

    #--------------------------------------------------------------------------------

    #calculate |E1| , |E2| , |E12|---------------------------------------------------
    E={}
    E12=0
    for l in layer:
        E[l]=0
        for n in layer[l]:
            for nei in node_l.get(n,set()):
                E[l]+= graph[n][nei].get('weight',1)
        E[l] = E[l]/2

    for n in node_c:
        for nei in node_c[n]:
            E12+=graph[n][nei].get('weight',1)
    E12 = E12/2
    #print(E12)
    #--------------------------------------------------------------------------------

    
    modularity=0    
    x1={}
    x2={}    
    
    for c in commu:
        x1[c]=0
        x2[c]=0
        modc_layer=0
        #print(len(intra_inter[c]))
        if len(intra_inter[c])>1:
            for l in layer:
                Aij=0.0
                hihj=0.0
                
                #compute summation Aij------------------------
                for n in commu[c]:
                    if n in layer[l]:
                        for nei in node_l.get(n,set()):
                            if nei in commu[c]:
                                Aij+=graph[n][nei].get('weight',1)
                #Aij = Aij/2

                #compute summation hihj--------------------------
                for n1 in commu[c]:
                    if n1 in layer[l]:
                        for n2 in commu[c]:
                            if n2 in layer[l]:
                                if(n1==n2):
                                    if(n1 not in node_l[n1]): 
                                        continue
                                hi = sum([graph[n1][nbr].get('weight',1) for nbr in node_l.get(n1,set())])
                                hj =sum([graph[n2][nbr].get('weight',1) for nbr in node_l.get(n2,set())])
                                hihj+= (hi*hj)
                #hihj = hihj/2
                #-------------------------------------------------
                try:
                    mod = (1.0/(2*E[l]))*(Aij - (hihj*1.0/(2*E[l])))
                except:
                    mod=0
                
                if f==1:# f is always 0
                    modc_layer+=mu*mod
                else:    
                    modc_layer+=mod
                    x1[c]+=mod
                #print ("modc_layer ",modc_layer)            

        #----------------------------------------------------------------------
            modc_couple=0
            Aij=0
            cicj=0
            
            #compute Aij------------------------------
            for n in commu[c]:
                if n in node_c:
                    for nei in node_c[n]:
                        if nei in commu[c]:
                            Aij+= graph[n][nei].get('weight',1)
            Aij = Aij/2

            #compute cicj-----------------------------
            tempmod=0.0
            for n1 in commu[c]:
                if(nodelayer[n1]==2):
                    continue
                for n2 in commu[c]:
                    
                    if(n1==n2 or nodelayer[n1]==nodelayer[n2] ) :
                        #If nodes in same layer, continue
                        continue
                    normi = 0;
                    normj=0;
                    if n1 in node_c:
                        ci = sum([graph[n1][nbr].get('weight',1) for nbr in node_c[n1]])

                        #extra penalty -> if a node is having coupling degree , then we penalize it for 
                        #            having link in its own layer with nodes that are not in the community
                        #extra_penalty = sum(graph[n1][nbr].get('weight',1) for nbr in node_l[n1] \
                        #                if nbr not in commu[c])

                        #if extra_penalty>0:
                        #    lay = nodelayer[n1]
                        #    normi+=2*E[lay]
                        #    ci+=extra_penalty
                        #    #print("for ",n1," norm+=",2*E[lay])

                        normi+=E12
                        #print("norm+=",E12)
                    else:
                        ci = sum([graph[n1][nbr].get('weight',1) for nbr in node_l.get(n1,set())])
                        lay = nodelayer[n1]
                        normi+=2*E[lay]
                        #print("for ",n1," norm+=",2*E[lay])
                    if n2 in node_c:
                        cj = sum([graph[n2][nbr].get('weight',1) for nbr in node_c[n2]])
                        
                        #extra penalty -> if a node is having coupling degree , then we penalize it for 
                        #            having link in its own layer with nodes that are not in the community
                        #extra_penalty = sum(graph[n2][nbr].get('weight',1) for nbr in node_l[n2] \
                        #                if nbr not in commu[c])
                        #if (extra_penalty>0):
                        #    lay = nodelayer[n2]
                        #    normj+=2*E[lay]
                        #    cj+=extra_penalty
                        #    print("for ",n2," norm+=",2*E[lay])

                        normj+=E12
                            #print(" norm+=",E12)
                    else:
                        cj = sum([graph[n2][nbr].get('weight',1) for nbr in node_l.get(n2,set())])
                        lay = nodelayer[n2]
                        normj+= 2*E[lay]
                        #print("for ",n2," norm+=",2*E[lay])
                    if(n1 in node_c[n2]):
                        aij=1
                    else:
                        aij=0
                    
                    try:
                        xyz=(aij*1.0/E12-(ci*cj*1.0)/(normi*1.0*normj))
                        tempmod+= xyz
                        #print("mp for node ",n1," ",n2," ",xyz)
                        #print("aij= ",aij," ci= ",ci, " cj= ",cj," normi= ",normi," normj= ",normj," E12= ",E12)
                        
                        #cicj += (ci*cj*1.0)/(norm*norm*1.0)
                        #print("ci = ",ci," cj = ",cj," norm = ",norm)
                    except:
                        tempmod+=0
                        #cicj +=0
            '''
            cicj = cicj/2

            try:
                mod = (Aij*1.0/E12 - (cicj))
            except:
                mod=0
            '''
            mod = tempmod
            if f==1:
                modc_couple+=2*(1-mu)*mod
            else:
                modc_couple+=mod
                x2[c]+=mod
            #print ("modc_couple ",modc_couple)            
            ##print "ha hh"

            #-----------------------------------------------------------------------
            modularity+=modc_layer+modc_couple
            
        else:
            l=list(intra_inter[c])
            l=l[0];
            Aij=0.0
            hihj=0.0
            
            #compute summation AIJ------------------------
            for n in commu[c]:
                for nei in node_l.get(n,set()):
                    if nei in commu[c]:
                        Aij+=graph[n][nei].get('weight',1)
            #Aij = Aij/2

            #compute summation hihj--------------------------
            for n1 in commu[c]:
                for n2 in commu[c]:
                    if(n1==n2):
                        if(n1 not in node_l[n1]):   #implies not a loop
                            continue
                    normi=0
                    normj=0
                    hi = sum([graph[n1][nbr].get('weight',1) for nbr in node_l.get(n1,set())])
                    ci = sum([graph[n1][nbr].get('weight',1) for nbr in node_c.get(n1,set())])
                    hj =sum([graph[n2][nbr].get('weight',1) for nbr in node_l.get(n2,set())])
                    cj = sum([graph[n2][nbr].get('weight',1) for nbr in node_c.get(n2,set())])
                    if(hi!=0):
                        normi+=2.0*E[l]
                    if(ci!=0):
                        normi+=E12*1.0
                    if(hj!=0):
                        normj+=2.0*E[l]
                    if(cj!=0):
                        normj+=E12*1.0
                    hihj+= (((hi+ci)*1.0/normi)*((hj+cj)*1.0/normj))
                    #hihj+= ((hi+ci)*(hj+cj))

            #hihj = hihj/2
            
            #-------------------------------------------------
            try:
                #norm = 1.0/(2*E[l] + E12)
                #mod = norm*(Aij*1.0 - (norm*hihj))
                mod = ((Aij*1.0)/(2.0*E[l]) - (hihj))

            except:
                mod=0
            
            if f==1: #f is always 0
                modc_layer+=mu*mod
            else:    
                modc_layer+=mod
                x1[c]+=mod
            #print("modc_layer",modc_layer)
            modularity+=modc_layer        
                                    
    return x1,x2    
    #return 0.333*modularity    



In [5]:
modctr = 0

def __modularity(commu, status, graph):
    global modctr
    modctr += 1
    #print("modularity called", modctr, "edgewt: ", [graph[1][nbr].get('weight',1) for nbr in graph[1]])
    #print("From modularity, node_c: ", status.node_c)
    #print("From modularity, node_l: ", status.node_l)

    layer=status.layer
    node_l=status.node_l
    node_c=status.node_c       
    top=status.top
    bot=status.bot
    edge_l=status.edge_l
    edge_c=status.edge_c
    couple=status.couple
    mu = status.mu
    
    #Construct a dict = {node1: layer, node2: layer......}
    nodelayer = {}
    for l in layer:
        for node in layer[l]:
            nodelayer[node] = l

    #calculate Intra_inter ----------------------------------------------------------
    f=0    
    intra_inter={}
    for c in commu:
        intra_inter[c]=set()
        
        for n in commu[c]:
            for l in layer:
                if n in layer[l]:
                    intra_inter[c].add(l)

    #--------------------------------------------------------------------------------

    #calculate |E1| , |E2| , |E12|---------------------------------------------------
    E={}
    E12=0
    for l in layer:
        E[l]=0
        for n in layer[l]:
            for nei in node_l.get(n,set()):
                E[l]+= graph[n][nei].get('weight',1)
        E[l] = E[l]/2

    for n in node_c:
        for nei in node_c[n]:
            E12+=graph[n][nei].get('weight',1)
    E12 = E12/2
    #print(E12)
    #--------------------------------------------------------------------------------

    
    modularity=0    
    x1={}
    x2={}    
    
    for c in commu:
        x1[c]=0
        x2[c]=0
        modc_layer=0
        #print(len(intra_inter[c]))
        if len(intra_inter[c])>1:
            for l in layer:
                Aij=0.0
                hihj=0.0
                
                #compute summation Aij------------------------
                for n in commu[c]:
                    if n in layer[l]:
                        for nei in node_l.get(n,set()):
                            if nei in commu[c]:
                                Aij+=graph[n][nei].get('weight',1)
                #Aij = Aij/2

                #compute summation hihj--------------------------
                for n1 in commu[c]:
                    if n1 in layer[l]:
                        for n2 in commu[c]:
                            if n2 in layer[l]:
                                if(n1==n2):
                                    if(n1 not in node_l[n1]): 
                                        continue
                                hi = sum([graph[n1][nbr].get('weight',1) for nbr in node_l.get(n1,set())])
                                hj =sum([graph[n2][nbr].get('weight',1) for nbr in node_l.get(n2,set())])
                                hihj+= (hi*hj)
                #hihj = hihj/2
                #-------------------------------------------------
                try:
                    mod = (1.0/(2*E[l]))*(Aij - (hihj*1.0/(2*E[l])))
                except:
                    mod=0
                
                if f==1:# f is always 0
                    modc_layer+=mu*mod
                else:    
                    modc_layer+=mod
                    x1[c]+=mod
                #print ("modc_layer ",modc_layer)            

        #----------------------------------------------------------------------
            modc_couple=0
            Aij=0
            cicj=0
            
            #compute Aij------------------------------
            for n in commu[c]:
                if n in node_c:
                    for nei in node_c[n]:
                        if nei in commu[c]:
                            Aij+= graph[n][nei].get('weight',1)
            Aij = Aij/2

            #compute cicj-----------------------------
            tempmod=0.0
            for n1 in commu[c]:
                if(nodelayer[n1]==2):
                    continue
                for n2 in commu[c]:
                    
                    if(n1==n2 or nodelayer[n1]==nodelayer[n2] ) :
                        #If nodes in same layer, continue
                        continue
                    normi = 0;
                    normj=0;
                    if n1 in node_c:
                        ci = sum([graph[n1][nbr].get('weight',1) for nbr in node_c[n1]])

                        #extra penalty -> if a node is having coupling degree , then we penalize it for 
                        #            having link in its own layer with nodes that are not in the community
                        #extra_penalty = sum(graph[n1][nbr].get('weight',1) for nbr in node_l[n1] \
                        #                if nbr not in commu[c])

                        #if extra_penalty>0:
                        #    lay = nodelayer[n1]
                        #    normi+=2*E[lay]
                        #    ci+=extra_penalty
                        #    #print("for ",n1," norm+=",2*E[lay])

                        normi+=E12
                        #print("norm+=",E12)
                    else:
                        ci = sum([graph[n1][nbr].get('weight',1) for nbr in node_l.get(n1,set())])
                        lay = nodelayer[n1]
                        if(ci>0):
                            normi+=2*E[lay]
                        #print("for ",n1," norm+=",2*E[lay])
                    if n2 in node_c:
                        cj = sum([graph[n2][nbr].get('weight',1) for nbr in node_c[n2]])
                        
                        #extra penalty -> if a node is having coupling degree , then we penalize it for 
                        #            having link in its own layer with nodes that are not in the community
                        #extra_penalty = sum(graph[n2][nbr].get('weight',1) for nbr in node_l[n2] \
                        #                if nbr not in commu[c])
                        #if (extra_penalty>0):
                        #    lay = nodelayer[n2]
                        #    normj+=2*E[lay]
                        #    cj+=extra_penalty
                        #    print("for ",n2," norm+=",2*E[lay])

                        normj+=E12
                            #print(" norm+=",E12)
                    else:
                        cj = sum([graph[n2][nbr].get('weight',1) for nbr in node_l.get(n2,set())])
                        lay = nodelayer[n2]
                        if(cj>0):
                            normj+= 2*E[lay]
                        #print("for ",n2," norm+=",2*E[lay])
                    if(n1 in node_c[n2]):
                        aij=1
                    else:
                        aij=0
                    
                    try:
                        xyz=(aij*1.0/E12-(ci*cj*1.0)/(normi*1.0*normj))
                        tempmod+= xyz
                        #print("m for node ",n1," ",n2," ",xyz)
                        #print("aij= ",aij," ci= ",ci, " cj= ",cj," normi= ",normi," normj= ",normj," E12= ",E12)
                        
                        #cicj += (ci*cj*1.0)/(norm*norm*1.0)
                        #print("ci = ",ci," cj = ",cj," norm = ",norm)
                    except:
                        tempmod+=0
                        #cicj +=0
            '''
            cicj = cicj/2

            try:
                mod = (Aij*1.0/E12 - (cicj))
            except:
                mod=0
            '''
            mod = tempmod
            if f==1:
                modc_couple+=2*(1-mu)*mod
            else:
                modc_couple+=mod
                x2[c]+=mod
            #print ("modc_couple ",modc_couple)            
            ##print "ha hh"

            #-----------------------------------------------------------------------
            modularity+=modc_layer+modc_couple
            
        else:
            l=list(intra_inter[c])
            l=l[0];
            Aij=0.0
            hihj=0.0
            
            #compute summation AIJ------------------------
            for n in commu[c]:
                for nei in node_l.get(n,set()):
                    if nei in commu[c]:
                        Aij+=graph[n][nei].get('weight',1)
            #Aij = Aij/2

            #compute summation hihj--------------------------
            for n1 in commu[c]:
                for n2 in commu[c]:
                    if(n1==n2):
                        if(n1 not in node_l[n1]):   #implies not a loop
                            continue
                    normi=0
                    normj=0
                    hi = sum([graph[n1][nbr].get('weight',1) for nbr in node_l.get(n1,set())])
                    ci = sum([graph[n1][nbr].get('weight',1) for nbr in node_c.get(n1,set())])
                    hj =sum([graph[n2][nbr].get('weight',1) for nbr in node_l.get(n2,set())])
                    cj = sum([graph[n2][nbr].get('weight',1) for nbr in node_c.get(n2,set())])
                    if(hi!=0):
                        normi+=2.0*E[l]
                    if(ci!=0):
                        normi+=E12*1.0
                    if(hj!=0):
                        normj+=2.0*E[l]
                    if(cj!=0):
                        normj+=E12*1.0
                    hihj+= (((hi+ci)*1.0/normi)*((hj+cj)*1.0/normj))
                    #hihj+= ((hi+ci)*(hj+cj))

            #hihj = hihj/2
            
            #-------------------------------------------------
            try:
                #norm = 1.0/(2*E[l] + E12)
                #mod = norm*(Aij*1.0 - (norm*hihj))
                mod = ((Aij*1.0)/(2.0*E[l]) - (hihj))

            except:
                mod=0
            
            if f==1: #f is always 0
                modc_layer+=mu*mod
            else:    
                modc_layer+=mod
                x1[c]+=mod
            #print("modc_layer",modc_layer)
            modularity+=modc_layer        
                                    
    ##print x1,x2    
    return 0.333*modularity    



In [6]:
modctr = 0

def __modularity1(commu, status, graph):
    global modctr
    modctr += 1
    
    layer=status.layer
    node_l=status.node_l
    node_c=status.node_c       
    top=status.top
    bot=status.bot
    edge_l=status.edge_l
    edge_c=status.edge_c
    couple=status.couple
    mu = status.mu
    

    #calculate |E| ---------------------------------------------------
    E=0
    for n in node_l:
        for nei in node_l[n]:
            E+=graph[n][nei].get('weight',1)

    for n in node_c:
        for nei in node_c[n]:
            E+=graph[n][nei].get('weight',1)
    E = E/2
    #--------------------------------------------------------------------------------

    
    modularity=0    
    
    #print(commu)
    
    for c in commu:
        
        #print("c = ",c)

        Aij=0.0
        hihj=0.0
        
        #compute summation AIJ------------------------
        for n in commu[c]:
            for nei in node_l.get(n,set()):
                if nei in commu[c]:
                    Aij+=graph[n][nei].get('weight',1)
                    #print("addingL ",graph[n][nei].get('weight',1), "for nodes ",n," & ",nei)
            for nei in node_c.get(n,set()):
                if nei in commu[c]:
                    Aij+=graph[n][nei].get('weight',1)
                    #print("addingC ",graph[n][nei].get('weight',1), "for nodes ",n," & ",nei)
        #Aij = Aij/2

        #compute summation hihj--------------------------
        for n1 in commu[c]:
            for n2 in commu[c]:
                if(n1==n2):
                    if(n1 not in node_l[n1] and n1 not in node_c[n1]):   #implies not a loop
                        continue
                
                hi = sum([graph[n1][nbr].get('weight',1) for nbr in node_l.get(n1,set())])
                ci = sum([graph[n1][nbr].get('weight',1) for nbr in node_c.get(n1,set())])
                hj =sum([graph[n2][nbr].get('weight',1) for nbr in node_l.get(n2,set())])
                cj = sum([graph[n2][nbr].get('weight',1) for nbr in node_c.get(n2,set())])
                
                #print("for nodes ",n1," and ",n2,"hi ci hj cj",hi," ",ci," ",hj," ",cj)
                
                temp= (((hi+ci)*(hj+cj)*1.0)/(2.0*E))
                hihj+=temp
                #print(temp)
                #hihj+= ((hi+ci)*(hj+cj))

        #hihj = hihj/2
        
        #-------------------------------------------------
        try:
            #print("aij = ",Aij," hi = ", hi+ci," hj= ",hi+cj," hihj = ",hihj," 2E= ",2.0*E)
            #norm = 1.0/(2*E[l] + E12)
            #mod = norm*(Aij*1.0 - (norm*hihj))
            mod = ((Aij*1.0)- (hihj))*1.0/(2.0*E)

        except:
            mod=0
        
        modularity=mod
                                    
    ##print x1,x2    
    return 0.333*modularity    



In [7]:

__PASS_MAX = -1
__MIN = 0.0000001
#modctr = 0


def is_multi_layer(e1, e2, node_c):
    if e2 in node_c and e1 in node_c[e2]:
        return True
    return False

def is_commu(e1, e2, commu):
    for c in commu:
        if e1 in commu[c] and e2 in commu[c]:
            return True
    return False

def partition_at_level(dendogram, level) :
    partition = dendogram[0].copy()
    for index in range(1, level + 1) :
        for node, community in partition.iteritems() :
            partition[node] = dendogram[index][community]
    return partition

def __renumber(dictionary) :
    count = 1
    #count = 0
    ret = dictionary.copy()
    new_values = dict([])

    for key in dictionary.keys() :
        value = dictionary[key]
        new_value = new_values.get(value, -1)
        if new_value == -1 :
            new_values[value] = count
            new_value = count
            count = count + 1
        ret[key] = new_value

    return ret

def _get_commu_dict(node2com):
    commu={}
    count = 1
    new_values = dict([])
    for n in node2com.keys():
        v = node2com[n]
        new_value = new_values.get(v, -1)
        if new_value == -1 :
            new_values[v] = count
            new_value = count
            commu[new_value] = set()
            count = count + 1
        commu[new_value].add(n)
    return commu



def __neighcom(node, graph, status) :
    weights = []
    for neighbor in graph[node]:
        if neighbor != node :
            neighborcom = status.node2com[neighbor]
            weights.append(neighborcom)
    return weights


#New __one_level function (from Raphael's code)
def __one_level(graph, status, status_list, level_count, verbose=0) :
    #print("graph edges: ",graph.edges(data = True))
    modif = True
    nb_pass_done = 0
    #p_temp = __renumber(status.node2com)
    p_temp = status.node2com
    
    status_list.append(p_temp)
    #cur_mod = __modularity(_get_commu_dict(status_list[-1]), status, graph)
    cur_mod = __modularity(_get_com_wise_nodes(status_list[-1]), status, graph)
    
    status_list.pop()
    new_mod = cur_mod
    #print "# id_node from_com to_com local_mod mod"
    while modif  and nb_pass_done != __PASS_MAX :
        cur_mod = new_mod
        modif = False
        nb_pass_done += 1
        cur_mod2 = cur_mod
        nodelist = graph.nodes()
        #for node in graph.nodes():
        for node in nodelist:
            com_node = status.node2com[node]
            neigh_communities = __neighcom(node, graph, status)
            status.node2com[node] = -1
            best_com = com_node
            best_increase = 0
            
            print("for node {0} neighcom are,{1}".format(node,neigh_communities))
            
            for com in neigh_communities:
            #for com in neigh_communities:

                temp_dict = {com:_get_com_wise_nodes(status.node2com)[com]}
                base_mod_of_community = __modularity(temp_dict, status, graph)

                status.node2com[node] = com

                #p_temp = __renumber(status.node2com)
                p_temp = status.node2com
                status_list.append(p_temp)
                
                #incr = __modularity(_get_commu_dict(status_list[-1]), status, graph) - cur_mod2

                temp_dict = {com:_get_com_wise_nodes(status_list[-1])[com]}
                incr = __modularity(temp_dict, status, graph) - base_mod_of_community
                
                print("moving ",node ," to ", com)
                #x1 = __modularity(temp_dict, status, graph)
                #print("mod of community: ",com ," is ",x1)
                x1,x2=  __modularityprint(temp_dict, status, graph)
                print("mod layer = ",x1," mod couple = ",x2, "incr = ", incr)
                print(" commu: ",p_temp)
                
                status_list.pop()

                if incr > best_increase :
                    best_increase = incr
                    best_com = com

                if(verbose): printsomeinformation(node, com, best_com, incr, status.node_l, status.node_c)

                status.node2com[node] = -1

            print("-------MOVING ",node," TO ",best_com," -----")
            status.node2com[node] = best_com
            
            #p_temp = __renumber(status.node2com)
            p_temp = status.node2com
            status_list.append(p_temp)
            #cur_mod2 =  __modularity(_get_commu_dict(status_list[-1]), status, graph)
            cur_mod2 =  __modularity(_get_com_wise_nodes(status_list[-1]), status, graph)
            status_list.pop()

            if best_com != com_node :
                modif = True
            
            if(verbose):
                pass
                #print("{0} {1} {2}".format(node, com_node, best_com))
                #nx.draw(graph, with_labels = True)
                #plt.show()

                '''p_temp2 = __renumber(status.node2com)
                status_list.append(p_temp2)
                incr =  __modularity(_get_commu_dict(partition_at_level(status_list, level_count)), status) - cur_mod2
                
                print node, com_node, best_com, incr, best_increase, __modularity(_get_commu_dict(partition_at_level(status_list, level_count)), status), cur_mod2
                status_list.pop()'''

        #p_temp = __renumber(status.node2com)
        p_temp = status.node2com
        status_list.append(p_temp)
        #new_mod = __modularity(_get_commu_dict(status_list[-1]), status, graph)
        new_mod = __modularity(_get_com_wise_nodes(status_list[-1]), status, graph)
        
        print("In __one_level new_mod: {0:.4f} cur_mod: {1:.4f}".format(new_mod,cur_mod))
        if(verbose): print("Status list[-1]: ",status_list[-1])
        
        status_list.pop()
        if new_mod - cur_mod < __MIN :
            break

#__modularity(_get_commu_dict(status_list[-1]), status)


def induced_graph_multilayer(partition, graph, status):
    
    print("Partiton created = ",partition)
    print("previously nodes = ",graph.nodes())
    new_layer =defaultdict(set)
    new_node_l=defaultdict(set)
    new_node_c=defaultdict(set)
    new_couple=defaultdict(set)
    layer = status.layer

    ret = nx.Graph()
    #id_extra_com = len(partition) + 1
    id_extra_com = max(partition.values()) + 1

    list_node_com = {}
    part = {}

    partition_rebuild = {}
    for id_node in partition:
        partition_rebuild.setdefault(partition[id_node], [])
        partition_rebuild[partition[id_node]].append(id_node)
    
    for id_com in partition_rebuild.keys():
        
        layer_node = {}
        is_top = False
        is_bot = False 
        for id_node in partition_rebuild[id_com]:
            if id_node in layer[1]:
                layer_node[id_node] = 1
                is_top = True
            else:
                layer_node[id_node] = 2
                is_bot = True

        if is_top and is_bot: # add two nodes into induced graph
            ret.add_node(id_com)
            ret.add_node(id_extra_com)
            #print("idcom: ", id_com, "idextracom: ",id_extra_com)
            #updating status
            new_layer[1].add(id_com)
            new_layer[2].add(id_extra_com)
            ###

            part[id_com] = id_com
            part[id_extra_com] = id_com # id_extra_com will be remain with id_com community

            for id_node in layer_node:
                if layer_node[id_node] == 1:
                    list_node_com[id_node] = id_com
                else:
                    list_node_com[id_node] = id_extra_com

            id_extra_com += 1

        else: # add one node into induced graph
            ret.add_node(id_com)
            #print("idcom: ", id_com)
            #updating status
            if(is_top): new_layer[1].add(id_com)
            elif(is_bot): new_layer[2].add(id_com)
            #######

            part[id_com] = id_com

            for id_node in layer_node:
                #part[id_node] = id_com
                list_node_com[id_node] = id_com

    for node1, node2, datas in graph.edges_iter(data = True):
        weight = datas.get("weight", 1)
        com1 = list_node_com[node1]
        com2 = list_node_com[node2]
        if(com1==com2):             # for selfloops add double edge weights
            weight *= 2
        w_prec = ret.get_edge_data(com1, com2, {"weight":0}).get("weight", 1)
        ret.add_edge(com1, com2, weight = w_prec + weight)

    #updating status
    for node1,node2 in ret.edges_iter():
        #if((node1 in layer[1] and node2 in layer[1]) or (node1 in layer[2] and node2 in layer[2])):
        if((node1 in new_layer[1] and node2 in new_layer[1]) or (node1 in new_layer[2] and node2 in new_layer[2])):
            #add to node_l
            new_node_l[node1].add(node2)
            new_node_l[node2].add(node1)
        else:
            #add to node_c
            new_node_c[node1].add(node2)
            new_node_c[node2].add(node1)

    #updating status
    new_couple[1]=set(ret.nodes())
        
    status.layer = new_layer
    status.couple = new_couple
    status.node_c = new_node_c
    status.node_l = new_node_l
    
    print("new nodes = ",ret.nodes())
    print("node_l: ",new_node_l)
    print("node_c: ",new_node_c)
    print("layer: ",new_layer)
    return ret, part,status

def louvain(graph, layer, node_l, node_c, top, bot, couple, edge_l, edge_c, mu) :
    current_graph = graph.copy()
    status = Status()
    
    status.layer=layer
    status.node_l=node_l
    status.node_c=node_c
    status.top=top
    status.bot=bot
    status.edge_l=edge_l
    status.edge_c=edge_c
    status.couple = couple
    status.mu = mu

    status.init(current_graph)

    mod = __modularity(_get_commu_dict(status.node2com), status, current_graph)
    status_list = list()
    level_count = 0
    
    __one_level(current_graph, status, status_list, level_count)
    #new_mod = __modularity(_get_commu_dict(status.node2com), status, current_graph)
    new_mod = __modularity(_get_com_wise_nodes(status.node2com), status, current_graph)
    partition = __renumber(status.node2com)
    
    status_list.append(partition)
    mod = new_mod
    print("mod now = ",mod," commu: ",partition)
    current_graph,part,status = induced_graph_multilayer(partition, current_graph,status)
    status.init(current_graph)

  
    while True :
        level_count+=1
        __one_level(current_graph, status, status_list, level_count, 1)
        
        partition = __renumber(status.node2com)
        status_list.append(partition)

        new_mod = __modularity(_get_com_wise_nodes(partition), status, current_graph)
        status_list.pop()
        
        if new_mod - mod < __MIN :
            print("In Louvain new_mod: {0:.4f} mod = {1:.4f} AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA".format(new_mod, mod))
            break
        status_list.append(partition)
        mod = new_mod
        #current_graph = induced_graph(partition, current_graph)
        current_graph, part,status = induced_graph_multilayer(partition, current_graph,status)
        #status.init(current_graph)
        status.init(current_graph, part)

        
    return status_list[:], mod

def computegtmod(filename):
    fnetwork = 0
    with open(filename+'_ml_network.pickle') as handle:
        fnetwork = pickle.load(handle)
    ml_network, layer, node_l, node_c, top, bot, couple, edge_l, edge_c, mu, commu = fnetwork
    
    status = Status()
    status.layer=layer
    status.node_l=node_l
    status.node_c=node_c
    status.top=top
    status.bot=bot
    status.edge_l=edge_l
    status.edge_c=edge_c
    status.couple = couple
    status.mu = mu
    mod = __modularity(commu, status, ml_network)
    return mod,commu

def getSeries(filename):
    fnetwork = 0
    with open(filename+'_ml_network.pickle') as handle:
        fnetwork = pickle.load(handle)
    ml_network, layer, node_l, node_c, top, bot, couple, edge_l, edge_c, mu, commu = fnetwork
    dendogram, mod = louvain(ml_network, layer, node_l, node_c, top, bot, couple, edge_l, edge_c, mu)
    return mod, dendogram
    
import os
import sys
import pickle

def write_commus_infile(network,detected_commu,gtcom):
	#writting the detected communities-------------------------------
    detected_communities_file = open(pathtowritecommu+str(network)+".pickle","w")
    pickle.dump((gtcom,detected_commu),detected_communities_file,2)
    detected_communities_file.close()
    #-----------------------------------------------------------------


#Comment following four lines if you want to run for all networks
#str2 = "./nets/network_0.2_1.0_0.05_1.0_0.0"
'''
str2 = "./nets/testnetwork2"

fnetwork = 0
filename = str2
with open(filename+'_ml_network.pickle') as handle:
    fnetwork = pickle.load(handle)
ml_network, layer, node_l, node_c, top, bot, couple, edge_l, edge_c, mu, commu = fnetwork

status = Status()
status.layer=layer
status.node_l=node_l
status.node_c=node_c
status.top=top
status.bot=bot
status.edge_l=edge_l
status.edge_c=edge_c
status.couple = couple
status.mu = mu

#status_list = [{1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 2, 7: 3, 8: 3, 9: 3, 10: 3, 11: 8, 12: 9, 13: 10, 14: 11, 15: 12, 16: 13, 17: 14, 18: 9}]
#status_list = [{1: 1, 2: 1, 6: 2, 7: 2, 8: 2, 9: 2, 10: 2}]
#status_list = [{1: 3, 2: 3, 6: 3, 7: 6, 8: 6, 9: 6, 10: 6, 11: 8, 12: 9, 13: 10, 14: 11, 15: 12, 16: 13, 17: 14, 18: 9}]
status_list = [{1: 2, 2: 2, 3: 2, 4: 3, 5: 5, 6:5 , 7: 3, 8: 3, 9: 3, 10: 3}]
#status_list = [{1: 2, 2: 2, 6: 5, 7: 4, 8: 5, 9: 6, 10: 7}]
#com = 5
#temp_dict = {com:_get_com_wise_nodes(status_list[-1])[com]}
temp_dict = _get_com_wise_nodes(status_list[-1])
mod = __modularity(temp_dict, status, ml_network)
print mod
'''
str2 = "./nets/testnetwork0"
modu, commus = getSeries(str2)
print("commus",commus)
print("Modularity: ", modu, "Communities: ",_get_com_wise_nodes(partition_at_level(commus, len(commus)-1)))
print("GT Mod: ",computegtmod(str2))


for node 1 neighcom are,[16, 2, 3, 4, 5]
('moving ', 1, ' to ', 16)
('mod layer = ', {16: 0.0}, ' mod couple = ', {16: 0.09876543209876543}, 'incr = ', 0.03288888888888889)
(' commu: ', {1: 16, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30})
('moving ', 1, ' to ', 2)
('mod layer = ', {2: 0.021867376618502535}, ' mod couple = ', {2: 0}, 'incr = ', 0.007281836413961344)
(' commu: ', {1: 2, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30})
('moving ', 1, ' to ', 3)
('mod layer = ', {3: 0.021867376618502535}, ' mod couple = ', {3: 0}, 'incr = ', 0.007281836413961344)
(' commu: ', {1: 3, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13

ZeroDivisionError: float division by zero